In [3]:
#load the train dataset - tiny shakespeare
!curl -L https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  4676k      0 --:--:-- --:--:-- --:--:-- 4674k


In [4]:
#read the data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print('lengt of the file', len(text))

lengt of the file 1115394


In [6]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
stoi = { c:i for i, c in enumerate(chars) }
itos = { i:c for i, c in enumerate(chars)}
s : str
encode = lambda s : [ stoi[x] for x in s] #encoder: Take a string and convert it do an integer 
it : list
decode  = lambda it : ''.join([itos[x] for x in it]) #decoder: take a list of integers, returns a string

print(encode('Frank the tank'))
print(decode(encode('Frank the tank')))

[18, 56, 39, 52, 49, 1, 58, 46, 43, 1, 58, 39, 52, 49]
Frank the tank


In [9]:
#encode the complete text
import torch
data : torch.tensor = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
#split of train and val data
n = int(0.9 * (len(data)))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
batch_size : int = 4 #number of parallel blocks 
block_size : int = 8 #number of characters per block
torch.manual_seed(1337)
def get_batch(split: str) -> tuple[torch.tensor, torch.tensor] : 
    data = train_data if split == 'train' else val_data
    ix : torch.tensor = torch.randint(len(data)-batch_size, (batch_size,))
    x : torch.Tuple[torch.Tensor, ] = torch.stack([data[i:i+block_size] for i in ix ])
    y : torch.Tuple[torch.Tensor, ] =  torch.stack([data[i+1:i+block_size+1] for i in ix ])
    return x, y
get_batch('val')

(tensor([[26, 16, 13, 10,  0, 27,  1, 42],
         [63,  6,  1, 57, 47, 56, 10,  0],
         [ 1, 46, 43, 56,  6,  0,  5, 41],
         [43, 63,  1, 40, 53, 53, 58,  7]]),
 tensor([[16, 13, 10,  0, 27,  1, 42, 43],
         [ 6,  1, 57, 47, 56, 10,  0, 57],
         [46, 43, 56,  6,  0,  5, 41, 59],
         [63,  1, 40, 53, 53, 58,  7, 46]]))

In [13]:
xb, yb = get_batch('train')
print(f'inputs: {xb.shape}')
print(xb)
print(f'targets: {yb.shape}')
print(yb)

for batch in range(batch_size): #batch dimension
    for t in range(block_size): # time dimension
        context = xb[batch, :t+1]
        print(f'input {context.tolist()} target is {yb[batch,t]}')


inputs: torch.Size([4, 8])
tensor([[43,  1, 39, 52,  1, 47, 52, 41],
        [53, 58, 46,  1, 40, 59, 56, 52],
        [53,  1, 40, 43,  1, 63, 53, 59],
        [ 1, 56, 43, 45, 39, 56, 42,  8]])
targets: torch.Size([4, 8])
tensor([[ 1, 39, 52,  1, 47, 52, 41, 46],
        [58, 46,  1, 40, 59, 56, 52,  8],
        [ 1, 40, 43,  1, 63, 53, 59, 56],
        [56, 43, 45, 39, 56, 42,  8,  0]])
input [43] target is 1
input [43, 1] target is 39
input [43, 1, 39] target is 52
input [43, 1, 39, 52] target is 1
input [43, 1, 39, 52, 1] target is 47
input [43, 1, 39, 52, 1, 47] target is 52
input [43, 1, 39, 52, 1, 47, 52] target is 41
input [43, 1, 39, 52, 1, 47, 52, 41] target is 46
input [53] target is 58
input [53, 58] target is 46
input [53, 58, 46] target is 1
input [53, 58, 46, 1] target is 40
input [53, 58, 46, 1, 40] target is 59
input [53, 58, 46, 1, 40, 59] target is 56
input [53, 58, 46, 1, 40, 59, 56] target is 52
input [53, 58, 46, 1, 40, 59, 56, 52] target is 8
input [53] target i

In [14]:
from torch.nn import functional as F
import torch.nn as nn
torch.manual_seed(1337)

In [65]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size : int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        #like in makemore each letter and the probality of the next token is stored in the embedding
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
#        print(f'idx.shape: {idx.shape}, targets.shape: {targets.shape}')
        logits = self.token_embedding_table(idx) #B-Batch, T - Time, C - Channel
        #print('logits:', logits)
        #why C - 65 ? This a are the class probalities for a certain character
        #logits has 4B, 8T, 65C <> Targets is 4B, 8T
        if(targets is None):
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
    #        print(logits.shape)
    #        print('targets', targets)
            targets = targets.view(B*T)

    #        print(targets.shape)
    #        print(targets)
            loss = F.cross_entropy(logits, targets)
#        print(loss)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #ids is (B,T) array of indicies in the current context
        for i in range(max_new_tokens):
            #predict the next tokens
            logits, loos = self(idx)
            #after prediction select the most proably next token. We use Softmax for prediction
            #focus on the last time step
            #print(f'logits shape: {logits.shape}')
            logits = logits[:,-1,:] #becomes (B,C)
            #print(f'After trans logits shape: {logits.shape}')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            #append sampled index to the running contect
            idx = torch.cat((idx, idx_next), dim=1) #(B, B+1)
        return idx            

In [66]:
m = BigramLanguageModel(vocab_size)
out = m(xb, yb)


In [67]:
testembed = nn.Embedding(10, 3)

In [68]:
testtorch = torch.LongTensor([9,1,2,3,5,6,4,2,1,4,5,])
testembed(testtorch)

tensor([[-0.5212,  0.6790, -0.5612],
        [-0.0085, -0.1171,  1.2822],
        [ 0.3471, -0.3520, -0.1387],
        [ 0.0459, -0.6033, -0.3042],
        [ 0.7840, -0.5271, -0.8057],
        [ 0.2760, -0.0936,  0.2109],
        [ 1.6794,  0.1600, -0.7944],
        [ 0.3471, -0.3520, -0.1387],
        [-0.0085, -0.1171,  1.2822],
        [ 1.6794,  0.1600, -0.7944],
        [ 0.7840, -0.5271, -0.8057]], grad_fn=<EmbeddingBackward0>)

In [69]:

next_tokens = m.generate(xb,2)
print(xb.shape)
print(next_tokens.shape)
print(next_tokens[1,:])
print(decode(next_tokens[1,:].tolist()))


torch.Size([32, 8])
torch.Size([32, 10])
tensor([46, 39, 58, 46,  1, 40, 56, 53, 57, 11])
hath bros;


In [70]:
next_tokens = m.generate(torch.zeros(1,1, dtype=torch.long),100)
print(decode(next_tokens[0].tolist()))


gnoqFi3MLY,csv,Xy&nUgF sAAZlMj
yIICMbFaSPSSc$qq;UQg&RTUZ'SlOs!DVdKlcPViMAuzJ'uKPCIBxJ?A:LpTILIwkKC&S


In [71]:
optimzer : torch.optim.AdamW= torch.optim.AdamW(m.parameters(), lr=1e-3)

In [79]:
batch_size = 32
for steps in range(1000):
    #sample a batch of data
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    
    optimzer.zero_grad(set_to_none=None)
    loss.backward()
    optimzer.step()
    
print(loss.item())
    

2.6962103843688965


In [80]:
next_tokens = m.generate(torch.zeros(1,1, dtype=torch.long),100)
print(decode(next_tokens[0].tolist()))


ILO,Vathy'xBEx teinlf lO:.

HNIN is w wroTryo;forveZ.
Fd.thjurshorshecAQCA'cok:Wechi&Dand.
; knthith


In [87]:
for i in m.parameters():
    print(i.shape)


torch.Size([65, 65])
